In [74]:
import ipywidgets as widgets
#from ipywidgets import interact, interactive
import subprocess

%matplotlib inline
import matplotlib.pyplot as plt
import random
import numpy as np

In [75]:
xmin = widgets.FloatText(
    value= -500.,
    description='$X_{min}$',
    disabled=False
)

ymin = widgets.FloatText(
    value= -500.,
    description='$Y_{min}$',
    disabled=False
)

zmin = widgets.FloatText(
    value= -500.,
    description='$Z_{min}$',
    disabled=False
)
    
xmax = widgets.FloatText(
    value= 500.,
    description='$X_{max}$',
    disabled=False
)

ymax = widgets.FloatText(
    value= 500.,
    description='$Y_{max}$',
    disabled=False
)

zmax = widgets.FloatText(
    value= 500,
    description='$Z_{max}$',
    disabled=False
)

tmax = widgets.BoundedFloatText(
    min = 0.,
    max = 100000000,
    value= 10000.,
    description='$Time_{max}$',
    disabled=False
)
    
xdelta = widgets.BoundedFloatText(
    min = 1.,
    value= 20.,
    description='$X_{delta}$',
    disabled=False
)

ydelta = widgets.BoundedFloatText(
    min = 1.,
    value= 20.,
    description='$Y_{delta}$',
    disabled=False
)

zdelta = widgets.BoundedFloatText(
    min = 1.,
    value= 20.,
    description='$Z_{delta}$',
    disabled=False
)
    
tdelta = widgets.BoundedFloatText(
    min = 0.01,
    value= 10.,
    description='$Time_{delta}$',
    disabled=False
)
x_row = widgets.HBox([xmin,xmax,xdelta])
y_row = widgets.HBox([ymin,ymax,ydelta])
z_row = widgets.HBox([zmin,zmax,zdelta])

omp_threads = widgets.BoundedIntText(
    value=8,
    min=1,
    step=1,
    description='# threads',
    disabled=False
)
prng_toggle = widgets.Checkbox(
    value=False,
    description='Seed PRNG',
    disabled=False
)
prng_seed = widgets.BoundedIntText(
    min = 1,
    value=13,
    description='Seed value',
    disabled=False
)
prng_row = widgets.HBox([prng_toggle, prng_seed])

#----- Output ------
output_dir_str = 'output'  # match the "value" of the widget below
def config_output_dir_cb(w):
    global output_dir_str
    output_dir_str = w.value
    print(output_dir_str)
    
output_dir = widgets.Text(
    value='output',
    description='Output Dir:',
)

output_dir.on_submit(config_output_dir_cb)

svg_toggle = widgets.Checkbox(
    value=True,
    description='.svg',
    disabled=False
)
svg_t0 = widgets.BoundedFloatText (
    min=0,
    value=0.0,
    description='$T_0$',
    disabled=False
)
svg_num_delT = widgets.BoundedIntText(
    min=1,
    value=5,
    description='Every $N^{th}$ time step',
    disabled=False
)

mcds_toggle = widgets.Checkbox(
    value=False,
    description='MultiCellDS',
    disabled=False
)
mcds_t0 = widgets.FloatText(
    value=0.0,
    description='$T_0$',
    disabled=True
)
mcds_num_delT = widgets.BoundedIntText(
    min=0,
    value=5,
    description='Every $N^{th}$ time step',
    disabled=True
)

#----------------------------
def read_config_file_cb(b):
    global pc4nanobio_config_xml
    import xml.etree.ElementTree as ET
    tree = ET.parse('default.xml')
    return

read_config_button = widgets.Button(
    description='Read config file',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Read XML',
)
read_config_file = widgets.Text(
    value='default.xml',
    description='',
    disabled=False
)
read_config_button.on_click(read_config_file_cb)

#----------------------------
def write_config_file_cb(b):
    print('writing config')

    import xml.etree.cElementTree as ET
    from lxml import etree

    root = ET.Element("root")
    user_details = ET.SubElement(root, "user_details")

    ET.SubElement(user_details, "PhysiCell_settings", name="version").text = "devel-version"
    ET.SubElement(user_details, "field2", name="asdfasd").text = "some vlaue2"

    tree = ET.ElementTree(root)
#    tree.write(write_config_file.value)
#    tree.write("test.xml")

    # TODO: verify can write to this filename
    tree.write(write_config_file.value)
     
    #<?xml version="1.0" encoding="UTF-8"?>
#<user_details />
#<PhysiCell_settings version="devel-version">
#        <domain>
    
    
write_config_button = widgets.Button(
    description='Write config file',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Generate XML',
)
write_config_button.on_click(write_config_file_cb)

write_config_file = widgets.Text(
    value='nanobio_settings.xml',
    description='',
    disabled=False
)

#----------------------------
run_output = widgets.Output()
run_output

def run_cb(b):
    global run_proc
    print('run sim...')
    with run_output:
        args = ['/Users/heiland/git/PhysiCell-nanobio-com-fork/trunk/src/PhysiCell-nanobio','nanobio_settings.xml']
        run_proc = subprocess.Popen(args)
    
run_button = widgets.Button(
    description='Run',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run simulation',
)
run_button.on_click(run_cb)

run_command = widgets.Text(
    value='PhysiCell-nanobio nanobio_settings.xml',
    description='',
    disabled=False
)

def kill_cb(b):
    global run_proc
    print('kill sim...')
    run_proc.terminate()
    
kill_button = widgets.Button(
    description='Kill',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Kill simulation',
)
kill_button.on_click(kill_cb)

read_config_row = widgets.HBox([read_config_button, read_config_file])
svg_output_row = widgets.HBox([svg_toggle, svg_t0, svg_num_delT])
mat_output_row = widgets.HBox([mcds_toggle, mcds_t0, mcds_num_delT])
write_config_row = widgets.HBox([write_config_button, write_config_file])
run_sim_row = widgets.HBox([run_button, run_command, kill_button])
config_tab = widgets.VBox([read_config_row, x_row,y_row,z_row,  tmax, omp_threads, prng_row,  
                           output_dir,svg_output_row,mat_output_row, write_config_row, run_sim_row, run_output])


#----------------------------------------------
import sys
import glob
import os
import xml.etree.ElementTree as ET
import math
join_our_list = "(Join/ask questions at https://groups.google.com/forum/#!forum/physicell-users)\n"

import matplotlib
import matplotlib.colors as mplc
import numpy as np  # if mpl was installed, numpy should have been too.
from collections import deque
import matplotlib.pyplot as plt

#current_idx = 0

#for idx in range(len(sys.argv)):
use_defaults = True
show_nucleus = 0
current_idx = 0
axes_min = 0.0
axes_max = 2000
axes_max = 1000
scale_radius = 1.0

svg_dir_str = '.'

def plot_svg(SVG):
  global current_idx, axes_max
  global svg_dir_str
#  dir = svg_dir.value
  fname = svg_dir_str + "/snapshot%08d.svg" % SVG

  if (os.path.isfile(fname) == False):
    print("File does not exist: ",fname)
    return

  xlist = deque()
  ylist = deque()
  rlist = deque()
  rgb_list = deque()

#  print('\n---- ' + fname + ':')
  tree = ET.parse(fname)
  root = tree.getroot()
#  print('--- root.tag ---')
#  print(root.tag)
#  print('--- root.attrib ---')
#  print(root.attrib)


#  print('--- child.tag, child.attrib ---')
  numChildren = 0
  for child in root:
#    print(child.tag, child.attrib)
#    print("keys=",child.attrib.keys())
    if use_defaults and ('width' in child.attrib.keys()):
      axes_max = float(child.attrib['width'])
#      print("--- found width --> axes_max =", axes_max)
    if child.text and "Current time" in child.text:
      svals = child.text.split()
      #title_str = "(" + str(current_idx) + ") Current time: " + svals[2] + "d, " + svals[4] + "h, " + svals[7] + "m"
      title_str = "Current time: " + svals[2] + "d, " + svals[4] + "h, " + svals[7] + "m"

#    print("width ",child.attrib['width'])
#    print('attrib=',child.attrib)
#    if (child.attrib['id'] == 'tissue'):
    if ('id' in child.attrib.keys()):
#      print('-------- found tissue!!')
      tissue_parent = child
      break

#  print('------ search tissue')
  cells_parent = None

  for child in tissue_parent:
#    print('attrib=',child.attrib)
    if (child.attrib['id'] == 'cells'):
#      print('-------- found cells, setting cells_parent')
      cells_parent = child
      break
    numChildren += 1


  num_cells = 0
#  print('------ search cells')
  for child in cells_parent:
#    print(child.tag, child.attrib)
#    print('attrib=',child.attrib)
    for circle in child:  # two circles in each child: outer + nucleus
    #  circle.attrib={'cx': '1085.59','cy': '1225.24','fill': 'rgb(159,159,96)','r': '6.67717','stroke': 'rgb(159,159,96)','stroke-width': '0.5'}
#      print('  --- cx,cy=',circle.attrib['cx'],circle.attrib['cy'])
      xval = float(circle.attrib['cx'])

      s = circle.attrib['fill']
#      print("s=",s)
#      print("type(s)=",type(s))
      if (s[0:3] == "rgb"):  # if an rgb string, e.g. "rgb(175,175,80)" 
        rgb = list(map(int, s[4:-1].split(",")))  
        rgb[:]=[x/255. for x in rgb]
      else:     # otherwise, must be a color name
        rgb_tuple = mplc.to_rgb(mplc.cnames[s])  # a tuple
        rgb = [x for x in rgb_tuple]

      # test for bogus x,y locations (rwh TODO: use max of domain?)
      too_large_val = 10000.
      if (math.fabs(xval) > too_large_val):
        print("bogus xval=",xval)
        break
      yval = float(circle.attrib['cy'])
      if (math.fabs(yval) > too_large_val):
        print("bogus xval=",xval)
        break

      rval = float(circle.attrib['r'])
#      if (rgb[0] > rgb[1]):
#        print(num_cells,rgb, rval)
      xlist.append(xval)
      ylist.append(yval)
      rlist.append(rval)
      rgb_list.append(rgb)

#     For .svg files with cells that *have* a nucleus, there will be a 2nd
      if (show_nucleus == 0):
        break

    num_cells += 1

#    if num_cells > 3:   # for debugging
#      print(fname,':  num_cells= ',num_cells," --- debug exit.")
#      sys.exit(1)
#      break

  #print(fname,':  num_cells= ',num_cells)

  xvals = np.array(xlist)
  yvals = np.array(ylist)
  rvals = np.array(rlist)
  rgbs =  np.array(rgb_list)
#print("xvals[0:5]=",xvals[0:5])
#print("rvals[0:5]=",rvals[0:5])
#  print("rvals.min, max=",rvals.min(),rvals.max())

  plt.cla()
  title_str += " (" + str(num_cells) + " agents)"
  plt.title(title_str)
  plt.xlim(axes_min,axes_max)
  plt.ylim(axes_min,axes_max)
  plt.scatter(xvals,yvals, s=rvals*scale_radius, c=rgbs)
#plt.xlim(0,2000)  # TODO - get these values from width,height in .svg at top
#plt.ylim(0,2000)
#  plt.pause(time_delay)
  plt.axis('equal')


# keep last plot displayed
#plt.ioff()
#plt.show()

def svg_dir_cb(w):
    global svg_dir_str
    svg_dir_str = w.value
    print(svg_dir_str)
    
svg_dir = widgets.Text(
    value='.',
    description='Directory:',
)
#svg_dir.observe(svg_dir_cb)
svg_dir.on_submit(svg_dir_cb)

svg_plot = widgets.interactive(plot_svg, SVG=(0, 480), continuous_update=False)
output = svg_plot.children[-1]
output.layout.height = '300px'
svg_plot

svg_play = widgets.Play(
#     interval=10,
    value=50,
    min=0,
    max=100,
    step=1,
    description="Press play",
    disabled=False
)
svg_slider = widgets.IntSlider()
widgets.jslink((svg_play, 'value'), (svg_slider, 'value'))
widgets.HBox([svg_play, svg_slider])

svg_tab = widgets.VBox([svg_dir, svg_plot, svg_play])
#---------------------

tabs = widgets.Tab(children=[config_tab, svg_tab])
tabs.set_title(0, 'Configure/Run')
tabs.set_title(1, 'SVG Plots')
widgets.VBox(children=[tabs])

VBox(children=(Tab(children=(VBox(children=(HBox(children=(Button(button_style='info', description='Read config file', style=ButtonStyle(), tooltip='Read XML'), Text(value='default.xml'))), HBox(children=(FloatText(value=-500.0, description='$X_{min}$'), FloatText(value=500.0, description='$X_{max}$'), BoundedFloatText(value=20.0, description='$X_{delta}$', min=1.0))), HBox(children=(FloatText(value=-500.0, description='$Y_{min}$'), FloatText(value=500.0, description='$Y_{max}$'), BoundedFloatText(value=20.0, description='$Y_{delta}$', min=1.0))), HBox(children=(FloatText(value=-500.0, description='$Z_{min}$'), FloatText(value=500.0, description='$Z_{max}$'), BoundedFloatText(value=20.0, description='$Z_{delta}$', min=1.0))), BoundedFloatText(value=10000.0, description='$Time_{max}$', max=100000000.0), BoundedIntText(value=8, description='# threads', min=1), HBox(children=(Checkbox(value=False, description='Seed PRNG'), BoundedIntText(value=13, description='Seed value', min=1))), Text(value='output', description='Output Dir:'), HBox(children=(Checkbox(value=True, description='.svg'), BoundedFloatText(value=0.0, description='$T_0$'), BoundedIntText(value=5, description='Every $N^{th}$ time step', min=1))), HBox(children=(Checkbox(value=False, description='MultiCellDS'), FloatText(value=0.0, description='$T_0$', disabled=True), BoundedIntText(value=5, description='Every $N^{th}$ time step', disabled=True))), HBox(children=(Button(button_style='success', description='Write config file', style=ButtonStyle(), tooltip='Generate XML'), Text(value='nanobio_settings.xml'))), HBox(children=(Button(button_style='success', description='Run', style=ButtonStyle(), tooltip='Run simulation'), Text(value='PhysiCell-nanobio nanobio_settings.xml'), Button(button_style='danger', description='Kill', style=ButtonStyle(), tooltip='Kill simulation'))), Output())), VBox(children=(Text(value='.', description='Directory:'), interactive(children=(IntSlider(value=240, description='SVG', max=480), Output(layout=Layout(height='300px'))), _dom_classes=('widget-interact',)), Play(value=50, description='Press play')))), _titles={'0': 'Configure/Run', '1': 'SVG Plots'}),))

run sim...
kill sim...


In [76]:
SVG=13
fname = svg_dir.value + "/snapshot%08d.svg" % SVG

In [77]:
fname

'./snapshot00000013.svg'